In [1]:
model_id = "facebook/opt-2.7b"

In [2]:
from transformers import GPTQConfig

quantization_config = GPTQConfig(
     bits=4, group_size=128, dataset="wikitext2", desc_act=False,
)

In [3]:
from transformers import AutoModelForCausalLM

quantized_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map='auto'
)

config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

/root/miniforge3/lib/python3.10/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:411: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, qweight, scales, qzeros, g_idx, bits, maxq):
/root/miniforge3/lib/python3.10/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:419: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/root/miniforge3/lib/python3.10/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)
CUDA extension not installed.
CUDA extension not installed.


pytorch_model.bin:   0%|          | 0.00/5.30G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.30G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

...

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


In [4]:
save_path = "./data/hf/models/quantized-opt-2.7b" 
quantized_model.save_pretrained(save_path)

In [5]:
loaded_model = AutoModelForCausalLM.from_pretrained(
    save_path, device_map='auto', torch_dtype='auto'
)

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [7]:
text = "Python is the best programming language."
inputs = tokenizer(text, return_tensors="pt").to(0)

out = loaded_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))

Python is the best programming language.
I'm not sure if you're being sarcastic or not, but I'm going to assume you're not.                                         


In [15]:
from transformers import pipeline

generator = pipeline(
    'text-generation', model=model_id, device=0, do_sample=True, num_return_sequences=3
)
print(*generator(text), sep='\n')

Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=21) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


{'generated_text': 'Python is the best programming language.\nI prefer Ruby.'}
{'generated_text': "Python is the best programming language.\nI hear it's nice to program in.\nIt is, but it is a bit difficult to use compared to other languages.\nWhat language is easier to use?\nProbably C++ or Java.  I have used them both, and I can definitely say that python is by far the most difficult of the three.\nI know C++ isn't as easy as python, but I find it to be fairly simple compared to other languages.\nI have done C++ and Java. I find Java much easier to use, and I believe it has more features.\nI find java a lot easier to use than c++.  C++ seems like a language that has many functions that are redundant.  I don't think there are many functions in java that are redundant.\nThis is also a matter of opinion, but I think C++ is easier to use.  The language is more structured, and the code is much more concise and readable than Java."}
{'generated_text': "Python is the best programming langua